### Import Libraries

In [2]:
import glob
import pandas as pd
import pickle
import numpy as np

### Import Data Files and Create Master Data Set

In [3]:
# Create list of all data files
myfiles = []
for each_file in glob.glob('scripts/*.csv'):
    myfiles.append(each_file)

In [4]:
# Check the length to see that all 122 files got appended
len(myfiles)

122

In [5]:
# Check results of myfiles
myfiles[0]

'scripts/s2e07.csv'

In [6]:
# Create data frame and loop through every file
all_scripts = pd.DataFrame(columns=['Character', 'Line', 'Line_Number', 'Episode'])
for file in myfiles:
    script = pd.read_csv(file)
    script['Line_Number'] = script.index
    script['Episode'] = file.replace('scripts/', '').replace('.csv', '')
    all_scripts = all_scripts.append(script)
    

In [7]:
# There are three two part episodes. Find halfway count in line so that they can be split
# I know this won't be the actual cut point and could cause some issues in the analysis but I'm 
# concerned the different sized documents could cause issues
s6e01_count = (all_scripts[ all_scripts['Episode'] == 's6e01']['Line'].nunique())//2
s6e20_count = (all_scripts[ all_scripts['Episode'] == 's6e20']['Line'].nunique())//2
s7e12_count = (all_scripts[ all_scripts['Episode'] == 's7e12']['Line'].nunique())//2
print(s6e01_count, s6e20_count, s7e12_count)

456 417 452


In [8]:
# Split two part episodes into separate parts
all_scripts['Episode_Split'] = np.where((all_scripts['Episode'] == 's6e01') & (all_scripts['Line_Number'] < s6e01_count), 's6e01p1',
                              np.where((all_scripts['Episode'] == 's6e01') & (all_scripts['Line_Number'] >= s6e01_count), 's6e01p2', 
                              np.where((all_scripts['Episode'] == 's6e20') & (all_scripts['Line_Number'] < s6e20_count), 's6e20p1',
                              np.where((all_scripts['Episode'] == 's6e20') & (all_scripts['Line_Number'] >= s6e20_count), 's6e20p2',
                              np.where((all_scripts['Episode'] == 's7e12') & (all_scripts['Line_Number'] < s7e12_count), 's7e12p1',
                              np.where((all_scripts['Episode'] == 's7e12') & (all_scripts['Line_Number'] >= s7e12_count), 's7e12p2', 
                                 all_scripts['Episode']))))))
                                       

In [9]:
all_scripts = all_scripts.sort_values(['Episode_Split', 'Line_Number'])

In [105]:
# Pickle data set
with open('all_scripts.pickle', 'wb') as to_write:
        pickle.dump(all_scripts, to_write)

In [106]:
# Open to check that pickle worked
with open('all_scripts.pickle','rb') as read_file:
    scripts = pickle.load(read_file)

scripts.head()

,Character,Line,Line_Number,Episode,Episode_Split
0,Leslie Knope,Hello.,0,s1e01,s1e01
1,Leslie Knope,Hi.,1,s1e01,s1e01
2,Leslie Knope,"My name is Leslie Knope, and I work for the Pa...",2,s1e01,s1e01
3,Leslie Knope,Can I ask you a few questions?,3,s1e01,s1e01
4,Leslie Knope,"Would you say that you are, ""Enjoying yourself...",4,s1e01,s1e01


<br>

### Create Episode Data Set

In [107]:
# Drop unnecessary columns
scripts_no_char = scripts.drop(columns = ['Character', 'Line_Number'])
scripts_no_char.head()

,Line,Episode,Episode_Split
0,Hello.,s1e01,s1e01
1,Hi.,s1e01,s1e01
2,"My name is Leslie Knope, and I work for the Pa...",s1e01,s1e01
3,Can I ask you a few questions?,s1e01,s1e01
4,"Would you say that you are, ""Enjoying yourself...",s1e01,s1e01


In [108]:
# Append text for each episode and drop duplicates
scripts_no_char['Episode_Text'] = scripts_no_char.groupby('Episode_Split')['Line'].transform(lambda x : ' '.join(x)) 
episodes = scripts_no_char.drop(columns = 'Line')
episodes.drop_duplicates(inplace = True)
episodes.reset_index(drop=True, inplace = True)
episodes.head()

,Episode,Episode_Split,Episode_Text
0,s1e01,s1e01,"Hello. Hi. My name is Leslie Knope, and I work..."
1,s1e02,s1e02,"Well, one of the funner things that we do here..."
2,s1e03,s1e03,"Okay, now, see, here's a good example of a pla..."
3,s1e04,s1e04,"So, we've been called out to this hiking trail..."
4,s1e05,s1e05,"In a town as old as Pawnee, there's a lot of h..."


In [109]:
# Check that it worked for 2 part episodes
episodes.tail(25)

,Episode,Episode_Split,Episode_Text
100,s6e10,s6e10,"Hello, I am Leslie Knope, and I am here to int..."
101,s6e11,s6e11,I've got to say that this report is quite impr...
102,s6e12,s6e12,"What do you mean, you can't? Look I am throwin..."
103,s6e13,s6e13,"""Leslie Knope and Ben Wyatt were married one y..."
104,s6e14,s6e14,"Okay, Ron. Enough's enough. Let's talk plans f..."
105,s6e15,s6e15,"Okay, I don't want to over-hype this, but pres..."
106,s6e16,s6e16,"Aw, babe, you've gone crazy. The only thing I ..."
107,s6e17,s6e17,God! Sorry! Sorry! What's happening? I can't h...
108,s6e18,s6e18,"I'd have to check, but I'm pretty sure we can ..."
109,s6e19,s6e19,So what do we got so far? We need big ticket i...


In [110]:
# Pickle data set
with open('episodes.pickle', 'wb') as to_write:
        pickle.dump(episodes, to_write)

In [112]:
# Reopen data set to check if it worked
with open('episodes.pickle','rb') as read_file:
    episodes = pickle.load(read_file)

episodes.head()

,Episode,Episode_Split,Episode_Text
0,s1e01,s1e01,"Hello. Hi. My name is Leslie Knope, and I work..."
1,s1e02,s1e02,"Well, one of the funner things that we do here..."
2,s1e03,s1e03,"Okay, now, see, here's a good example of a pla..."
3,s1e04,s1e04,"So, we've been called out to this hiking trail..."
4,s1e05,s1e05,"In a town as old as Pawnee, there's a lot of h..."


<br>

### Create Leslie Episode Data Set

In [117]:
# Filter to just Leslie's lines
leslie = all_scripts[ all_scripts['Character'] == 'Leslie Knope' ]
leslie.head()

,Character,Line,Line_Number,Episode,Episode_Split
0,Leslie Knope,Hello.,0,s1e01,s1e01
1,Leslie Knope,Hi.,1,s1e01,s1e01
2,Leslie Knope,"My name is Leslie Knope, and I work for the Pa...",2,s1e01,s1e01
3,Leslie Knope,Can I ask you a few questions?,3,s1e01,s1e01
4,Leslie Knope,"Would you say that you are, ""Enjoying yourself...",4,s1e01,s1e01


In [118]:
# Append all the text from each episode and drop duplicates
leslie['Episode_Text'] = leslie.groupby('Episode_Split')['Line'].transform(lambda x : ' '.join(x)) 
leslie_episodes = leslie.drop(columns = ['Line', 'Line_Number'])
leslie_episodes.drop_duplicates(inplace = True)
leslie_episodes.reset_index(drop=True, inplace = True)
leslie_episodes.head()

<ipython-input-118-0e88844499bc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  leslie['Episode_Text'] = leslie.groupby('Episode_Split')['Line'].transform(lambda x : ' '.join(x))


,Character,Episode,Episode_Split,Episode_Text
0,Leslie Knope,s1e01,s1e01,"Hello. Hi. My name is Leslie Knope, and I work..."
1,Leslie Knope,s1e02,s1e02,"Well, one of the funner things that we do here..."
2,Leslie Knope,s1e03,s1e03,The Parks Department has so many programs. Jer...
3,Leslie Knope,s1e04,s1e04,"I don't believe it. Oh, my God. It's real. Hey..."
4,Leslie Knope,s1e05,s1e05,"In a town as old as Pawnee, there's a lot of h..."


In [119]:
# Pickle data set
with open('Leslie_all.pickle', 'wb') as to_write:
        pickle.dump(leslie_episodes, to_write)

In [120]:
# Reopen data set to check if it worked
with open('Leslie_all.pickle','rb') as read_file:
    leslie_episodes = pickle.load(read_file)

leslie_episodes.head()

,Character,Episode,Episode_Split,Episode_Text
0,Leslie Knope,s1e01,s1e01,"Hello. Hi. My name is Leslie Knope, and I work..."
1,Leslie Knope,s1e02,s1e02,"Well, one of the funner things that we do here..."
2,Leslie Knope,s1e03,s1e03,The Parks Department has so many programs. Jer...
3,Leslie Knope,s1e04,s1e04,"I don't believe it. Oh, my God. It's real. Hey..."
4,Leslie Knope,s1e05,s1e05,"In a town as old as Pawnee, there's a lot of h..."


<br>

### Create Ron Episode Data Set

In [123]:
# Filter to just Ron's lines
ron = all_scripts[ all_scripts['Character'] == 'Ron Swanson' ]
ron.head()

,Character,Line,Line_Number,Episode,Episode_Split
38,Ron Swanson,Tonight is our next monthly community outreach...,38,s1e01,s1e01
40,Ron Swanson,That is tonight.,40,s1e01,s1e01
42,Ron Swanson,Leslie will be running it.,42,s1e01,s1e01
43,Ron Swanson,And we need one more.,43,s1e01,s1e01
44,Ron Swanson,Who wants in?,44,s1e01,s1e01


In [124]:
# Append all the text from each episode and drop duplicates
ron['Episode_Text'] = ron.groupby('Episode_Split')['Line'].transform(lambda x : ' '.join(x)) 
ron_episodes = ron.drop(columns = ['Line', 'Line_Number'])
ron_episodes.drop_duplicates(inplace = True)
ron_episodes.reset_index(drop=True, inplace = True)
ron_episodes.head()

<ipython-input-124-fdbad8a048d0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ron['Episode_Text'] = ron.groupby('Episode_Split')['Line'].transform(lambda x : ' '.join(x))


,Character,Episode,Episode_Split,Episode_Text
0,Ron Swanson,s1e01,s1e01,Tonight is our next monthly community outreach...
1,Ron Swanson,s1e02,s1e02,"Uh, sure, Paul. What can I do for you? Yeah, a..."
2,Ron Swanson,s1e03,s1e03,"No comment. Hey, Haverford, maybe one day you'..."
3,Ron Swanson,s1e04,s1e04,Go to jail? What's going on? Put it in an emai...
4,Ron Swanson,s1e05,s1e05,The only reason anybody's going to this thing ...


In [125]:
# Pickle Data Set
with open('Ron_all.pickle', 'wb') as to_write:
        pickle.dump(ron_episodes, to_write)

In [126]:
# Reopen data set to check if it worked
with open('Ron_all.pickle','rb') as read_file:
    ron_episodes = pickle.load(read_file)

ron_episodes.head()

,Character,Episode,Episode_Split,Episode_Text
0,Ron Swanson,s1e01,s1e01,Tonight is our next monthly community outreach...
1,Ron Swanson,s1e02,s1e02,"Uh, sure, Paul. What can I do for you? Yeah, a..."
2,Ron Swanson,s1e03,s1e03,"No comment. Hey, Haverford, maybe one day you'..."
3,Ron Swanson,s1e04,s1e04,Go to jail? What's going on? Put it in an emai...
4,Ron Swanson,s1e05,s1e05,The only reason anybody's going to this thing ...


<br>

### Create Leslie and Ron Data Set

In [11]:
# Filter to Leslie's and Ron's Lines
leslie_ron = all_scripts[ all_scripts['Character'].isin(['Leslie Knope', 'Ron Swanson']) ]
leslie_ron.reset_index(inplace = True)
leslie_ron.head()

,index,Character,Line,Line_Number,Episode,Episode_Split
0,0,Leslie Knope,Hello.,0,s1e01,s1e01
1,1,Leslie Knope,Hi.,1,s1e01,s1e01
2,2,Leslie Knope,"My name is Leslie Knope, and I work for the Pa...",2,s1e01,s1e01
3,3,Leslie Knope,Can I ask you a few questions?,3,s1e01,s1e01
4,4,Leslie Knope,"Would you say that you are, ""Enjoying yourself...",4,s1e01,s1e01


In [12]:
# Create variables for line number difference and whether or not there was a character difference
leslie_ron['Line_Diff'] = leslie_ron.Line_Number  - leslie_ron.Line_Number.shift(1)
leslie_ron['Character_Change'] = leslie_ron['Character'].shift(1, fill_value=leslie_ron['Character'].head(1)) != leslie_ron['Character']
leslie_ron.head()

<ipython-input-12-01df62a537e6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  leslie_ron['Line_Diff'] = leslie_ron.Line_Number  - leslie_ron.Line_Number.shift(1)
<ipython-input-12-01df62a537e6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  leslie_ron['Character_Change'] = leslie_ron['Character'].shift(1, fill_value=leslie_ron['Character'].head(1)) != leslie_ron['Character']


,index,Character,Line,Line_Number,Episode,Episode_Split,Line_Diff,Character_Change
0,0,Leslie Knope,Hello.,0,s1e01,s1e01,NaN,False
1,1,Leslie Knope,Hi.,1,s1e01,s1e01,1,False
2,2,Leslie Knope,"My name is Leslie Knope, and I work for the Pa...",2,s1e01,s1e01,1,False
3,3,Leslie Knope,Can I ask you a few questions?,3,s1e01,s1e01,1,False
4,4,Leslie Knope,"Would you say that you are, ""Enjoying yourself...",4,s1e01,s1e01,1,False


In [13]:
# I need to append lines that were spoken in a row by the same character together. The code below creates a 
# 'Group' value that can be used to tell which lines to append. If the character changes or the next line number 
# isn't the line number after the current one, it puts that line into a new group

inc = 1

for index, row in leslie_ron.iterrows():
    if row['Character_Change'] == 1 or row['Line_Diff'] > 1:
        inc += 1
    test_val = inc
    leslie_ron.at[index,'Group'] = test_val

leslie_ron.head(10)

/Users/madelinevossbrinck/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/madelinevossbrinck/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,index,Character,Line,Line_Number,Episode,Episode_Split,Line_Diff,Character_Change,Group
0,0,Leslie Knope,Hello.,0,s1e01,s1e01,NaN,False,1.0
1,1,Leslie Knope,Hi.,1,s1e01,s1e01,1,False,1.0
2,2,Leslie Knope,"My name is Leslie Knope, and I work for the Pa...",2,s1e01,s1e01,1,False,1.0
3,3,Leslie Knope,Can I ask you a few questions?,3,s1e01,s1e01,1,False,1.0
4,4,Leslie Knope,"Would you say that you are, ""Enjoying yourself...",4,s1e01,s1e01,1,False,1.0
5,5,Leslie Knope,I'm gonna put a lot of fun.,5,s1e01,s1e01,1,False,1.0
6,7,Leslie Knope,"Sir, this is a children's slide.",7,s1e01,s1e01,2,False,2.0
7,8,Leslie Knope,You're not allowed to sleep in here.,8,s1e01,s1e01,1,False,2.0
8,10,Leslie Knope,"You know, when I first tell people that I work...",10,s1e01,s1e01,2,False,3.0
9,11,Leslie Knope,"""The government.""",11,s1e01,s1e01,1,False,3.0


In [14]:
# Get min and max line number for group of lines spoken in a row by same character and then append lines in each
# group together
leslie_ron['Max_Line'] = leslie_ron.groupby('Group')['Line_Number'].transform(max)
leslie_ron['Min_Line'] = leslie_ron.groupby('Group')['Line_Number'].transform(min)
leslie_ron['Episode_Text'] = leslie_ron.groupby('Group')['Line'].transform(lambda x : ' '.join(x))
leslie_ron.head()

<ipython-input-14-301b33571fed>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  leslie_ron['Max_Line'] = leslie_ron.groupby('Group')['Line_Number'].transform(max)
<ipython-input-14-301b33571fed>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  leslie_ron['Min_Line'] = leslie_ron.groupby('Group')['Line_Number'].transform(min)
<ipython-input-14-301b33571fed>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



,index,Character,Line,Line_Number,Episode,Episode_Split,Line_Diff,Character_Change,Group,Max_Line,Min_Line,Episode_Text
0,0,Leslie Knope,Hello.,0,s1e01,s1e01,NaN,False,1.0,5,0,"Hello. Hi. My name is Leslie Knope, and I work..."
1,1,Leslie Knope,Hi.,1,s1e01,s1e01,1,False,1.0,5,0,"Hello. Hi. My name is Leslie Knope, and I work..."
2,2,Leslie Knope,"My name is Leslie Knope, and I work for the Pa...",2,s1e01,s1e01,1,False,1.0,5,0,"Hello. Hi. My name is Leslie Knope, and I work..."
3,3,Leslie Knope,Can I ask you a few questions?,3,s1e01,s1e01,1,False,1.0,5,0,"Hello. Hi. My name is Leslie Knope, and I work..."
4,4,Leslie Knope,"Would you say that you are, ""Enjoying yourself...",4,s1e01,s1e01,1,False,1.0,5,0,"Hello. Hi. My name is Leslie Knope, and I work..."


In [15]:
# Drop unnecessary columns
leslie_ron.drop(columns = ['index', 'Line', 'Line_Number', 'Line_Diff', 'Character_Change'], inplace = True)
leslie_ron.head()

/Users/madelinevossbrinck/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Character,Episode,Episode_Split,Group,Max_Line,Min_Line,Episode_Text
0,Leslie Knope,s1e01,s1e01,1.0,5,0,"Hello. Hi. My name is Leslie Knope, and I work..."
1,Leslie Knope,s1e01,s1e01,1.0,5,0,"Hello. Hi. My name is Leslie Knope, and I work..."
2,Leslie Knope,s1e01,s1e01,1.0,5,0,"Hello. Hi. My name is Leslie Knope, and I work..."
3,Leslie Knope,s1e01,s1e01,1.0,5,0,"Hello. Hi. My name is Leslie Knope, and I work..."
4,Leslie Knope,s1e01,s1e01,1.0,5,0,"Hello. Hi. My name is Leslie Knope, and I work..."


In [17]:
# Remove duplicates
leslie_ron.drop_duplicates(inplace = True)
leslie_ron.head()

<ipython-input-17-bd812e8be26a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  leslie_ron.drop_duplicates(inplace = True)


,Character,Episode,Episode_Split,Group,Max_Line,Min_Line,Episode_Text
0,Leslie Knope,s1e01,s1e01,1.0,5,0,"Hello. Hi. My name is Leslie Knope, and I work..."
6,Leslie Knope,s1e01,s1e01,2.0,8,7,"Sir, this is a children's slide. You're not al..."
8,Leslie Knope,s1e01,s1e01,3.0,18,10,"You know, when I first tell people that I work..."
17,Leslie Knope,s1e01,s1e01,4.0,20,20,Do you want to come this way?
18,Leslie Knope,s1e01,s1e01,5.0,22,22,"Okay, we're gonna need you to get out."


In [19]:
# Create variables to tell if previous and next lines are subsequent or if there is a gap
# The idea is that if the max of the previous is the line before the min of the current or max of current is 
# the line before the min of the next, the lines were spoken subsequently and are part of the same conversation.
# This is definitely not foolproof, but was the best I could think of without going through entire script manually
leslie_ron['Prev_Line_Diff'] = leslie_ron.Min_Line  - leslie_ron.Max_Line.shift(1)
leslie_ron['Next_Line_Diff'] = leslie_ron.Min_Line.shift(-1) - leslie_ron.Max_Line
leslie_ron.head()

<ipython-input-19-c8530a6a5286>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  leslie_ron['Prev_Line_Diff'] = leslie_ron.Min_Line  - leslie_ron.Max_Line.shift(1)
<ipython-input-19-c8530a6a5286>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  leslie_ron['Next_Line_Diff'] = leslie_ron.Min_Line.shift(-1) - leslie_ron.Max_Line


,Character,Episode,Episode_Split,Group,Max_Line,Min_Line,Episode_Text,Prev_Line_Diff,Next_Line_Diff
0,Leslie Knope,s1e01,s1e01,1.0,5,0,"Hello. Hi. My name is Leslie Knope, and I work...",NaN,2.0
6,Leslie Knope,s1e01,s1e01,2.0,8,7,"Sir, this is a children's slide. You're not al...",2.0,2.0
8,Leslie Knope,s1e01,s1e01,3.0,18,10,"You know, when I first tell people that I work...",2.0,2.0
17,Leslie Knope,s1e01,s1e01,4.0,20,20,Do you want to come this way?,2.0,2.0
18,Leslie Knope,s1e01,s1e01,5.0,22,22,"Okay, we're gonna need you to get out.",2.0,2.0


In [234]:
# Keep only anything where the lines are subsequent of one another
leslie_ron_convo = leslie_ron[ (leslie_ron['Prev_Line_Diff'] == 1.0) | (leslie_ron['Next_Line_Diff'] == 1.0)]
leslie_ron_convo.head(10)

,Character,Episode,Episode_Split,Group,Max_Line,Min_Line,Episode_Text,Agg_Line_Diff,Next_Line_Diff,Prev_Line_Diff
21,Leslie Knope,s1e01,s1e01,7.0,37,27,"Here we go! Okay, wake up. Here we go. Out of ...",-1.0,1.0,2.0
32,Ron Swanson,s1e01,s1e01,8.0,38,38,Tonight is our next monthly community outreach...,-1.0,1.0,1.0
33,Leslie Knope,s1e01,s1e01,9.0,39,39,And that is tonight.,-1.0,1.0,1.0
34,Ron Swanson,s1e01,s1e01,10.0,40,40,That is tonight.,-1.0,1.0,1.0
35,Leslie Knope,s1e01,s1e01,11.0,41,41,Right.,-1.0,1.0,1.0
36,Ron Swanson,s1e01,s1e01,12.0,45,42,Leslie will be running it. And we need one mor...,-2.0,2.0,1.0
174,Leslie Knope,s1e01,s1e01,64.0,334,334,"Ron, please.",-1.0,1.0,3.0
175,Ron Swanson,s1e01,s1e01,65.0,336,335,No. No way.,-1.0,1.0,1.0
177,Leslie Knope,s1e01,s1e01,66.0,340,337,"Come on, Ron. I've been a loyal foot soldier. ...",-1.0,1.0,1.0
181,Ron Swanson,s1e01,s1e01,67.0,341,341,Is that a travel pillow around your neck?,-1.0,1.0,1.0


In [237]:
# Only keep necessary columns
leslie_ron_convo_final = leslie_ron_convo[['Character', 'Episode', 'Episode_Split', 'Episode_Text']]
leslie_ron_convo_final.reset_index(inplace = True)
leslie_ron_convo_final

,index,Character,Episode,Episode_Split,Episode_Text
0,21,Leslie Knope,s1e01,s1e01,"Here we go! Okay, wake up. Here we go. Out of ..."
1,32,Ron Swanson,s1e01,s1e01,Tonight is our next monthly community outreach...
2,33,Leslie Knope,s1e01,s1e01,And that is tonight.
3,34,Ron Swanson,s1e01,s1e01,That is tonight.
4,35,Leslie Knope,s1e01,s1e01,Right.
...,...,...,...,...,...
2156,22920,Ron Swanson,s7e12,s7e12p2,"Well, I have to make a couple more adjustments."
2157,22921,Leslie Knope,s7e12,s7e12p2,"Babe, I don't want to seem dramatic, but I hav..."
2158,23068,Leslie Knope,s7e12,s7e12p2,A [bleep] library?
2159,23069,Ron Swanson,s7e12,s7e12p2,"Yeah, that should do it. The swing is fixed!"


In [238]:
# Pickle data source
with open('Leslie_Ron_Convo.pickle', 'wb') as to_write:
        pickle.dump(leslie_ron_convo_final, to_write)

<br>

### Leslie and Ron Mention Each Other Data Set

In [22]:
# The idea behind this data set was to try to isolate any line where Leslie says Ron's name or Ron says Leslie's
# name and run the sentiment analysis on that. There weren't enough lines to get any meaningful results, so it 
# was not used in the presentation
leslie_ron_mention = all_scripts[ all_scripts['Character'].isin(['Leslie Knope', 'Ron Swanson']) ]
leslie_ron_mention.reset_index(inplace = True)
leslie_ron_mention.head()

,index,Character,Line,Line_Number,Episode,Episode_Split
0,0,Leslie Knope,Hello.,0,s1e01,s1e01
1,1,Leslie Knope,Hi.,1,s1e01,s1e01
2,2,Leslie Knope,"My name is Leslie Knope, and I work for the Pa...",2,s1e01,s1e01
3,3,Leslie Knope,Can I ask you a few questions?,3,s1e01,s1e01
4,4,Leslie Knope,"Would you say that you are, ""Enjoying yourself...",4,s1e01,s1e01


In [23]:
# Create variables for when they mention each other
leslie_ron_mention['Leslie_Mentions_Ron'] = np.where((leslie_ron_mention['Line'].str.contains('Ron', 'ron')) & (leslie_ron_mention['Character'] == 'Leslie Knope'), True, False)
leslie_ron_mention['Ron_Mentions_Leslie'] = np.where((leslie_ron_mention['Line'].str.contains('Leslie', 'leslie')) & (leslie_ron_mention['Character'] == 'Ron Swanson'), True, False)
leslie_ron_mention.head()

<ipython-input-23-44f89e41efeb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  leslie_ron_mention['Leslie_Mentions_Ron'] = np.where((leslie_ron_mention['Line'].str.contains('Ron', 'ron')) & (leslie_ron_mention['Character'] == 'Leslie Knope'), True, False)
<ipython-input-23-44f89e41efeb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  leslie_ron_mention['Ron_Mentions_Leslie'] = np.where((leslie_ron_mention['Line'].str.contains('Leslie', 'leslie')) & (leslie_ron_mention['Character'] == 'Ron Swanson'), True

,index,Character,Line,Line_Number,Episode,Episode_Split,Leslie_Mentions_Ron,Ron_Mentions_Leslie
0,0,Leslie Knope,Hello.,0,s1e01,s1e01,False,False
1,1,Leslie Knope,Hi.,1,s1e01,s1e01,False,False
2,2,Leslie Knope,"My name is Leslie Knope, and I work for the Pa...",2,s1e01,s1e01,False,False
3,3,Leslie Knope,Can I ask you a few questions?,3,s1e01,s1e01,False,False
4,4,Leslie Knope,"Would you say that you are, ""Enjoying yourself...",4,s1e01,s1e01,False,False


In [247]:
# Pickle Data Set
with open('Leslie_Ron_Mention.pickle', 'wb') as to_write:
        pickle.dump(leslie_ron_mention, to_write)